# Hadronic Baseline Calculation

## Role in the Project
This script performs the preliminary calibration phase for the simulation pipeline. Its specific purpose is to calculate the **maximum stable mass** for each "pure" (unmodified) hadronic Equation of State (EoS) available in the project's library.

These baseline values are essential for the subsequent data generation process. They serve as the reference standards for the **homologous scaling** technique, which allows the physics engine to mathematically transform a known parent model to hit a specific target mass range (e.g., ensuring a generated star can support $2.0 M_{\odot}$). Without these pre-calculated baselines, the "Smart Inverse Sampling" strategy used to generate diverse hadronic stars would not be possible.

## Physics and Equations

### The Maximum Mass Limit
For any given Equation of State, there exists a unique sequence of neutron star configurations parameterized by central pressure. As central pressure increases, the mass of the star increases until it reaches a maximum value, $M_{max}$. Beyond this point, the star becomes unstable to gravitational collapse (forming a black hole).

The TOV (Tolman–Oppenheimer–Volkoff) equilibrium solutions define this sequence. The maximum mass corresponds to the turning point in the Mass-Radius relation:

$$
\frac{dM}{dR} = 0 \quad \text{and} \quad \frac{dM}{d\epsilon_c} = 0
$$

### Homologous Scaling Reference
The project utilizes a scaling transformation to generate new synthetic EoS curves from existing parent models. If a parent model produces a maximum mass $M_{base}$, a new model can be generated by scaling the pressure $P \to \alpha P$ and energy density $\epsilon \to \alpha \epsilon$.

The maximum mass of the new scaled model, $M_{target}$, is related to the baseline by:

$$
M_{target} \approx \frac{1}{\sqrt{\alpha}} M_{base}
$$

To solve for the required scaling factor $\alpha$ needed to produce a star of a specific target mass, $M_{base}$ must be known. This script computes $M_{base}$ for every analytic model in the library (e.g., SLy, APR, MDI).

## Algorithm and Calculations

1.  **Library Retrieval**
    The collection of analytic hadronic EoS functions is loaded. This includes the pressure-energy density relations and sound speed derivatives for models derived from nuclear theory.

2.  **Iterative Solver**
    For each unique model in the library:
    *   The model is configured as a "pure" fluid (mixing weight $w=1.0$) with no scaling ($\alpha=1.0$).
    *   The appropriate crust-core transition pressure is selected (standard or model-specific).
    *   The TOV integrator is executed over a wide range of central pressures to generate the full stellar sequence.
    *   The maximum mass $M_{max}$ achieved in the sequence is identified and recorded.

3.  **Validation**
    Only models capable of supporting a stable star with $M > 1.0 M_{\odot}$ are retained in the baseline dictionary.

## Inputs and Outputs

### Inputs
*   **Analytic EoS Library**: The mathematical definitions of the pressure-energy density relations for all supported hadronic models.
*   **Physical Constants**: Numerical parameters for the crust transition and integration limits.

### Outputs
*   **Baseline Dictionary**: A mapping structure where keys are model names (e.g., "SLy", "MDI-2") and values are their corresponding maximum stable masses in solar masses ($M_{\odot}$).

## Connection to the Overall Workflow
This script is the **first computational step** in the pipeline. It is executed once at the beginning of the workflow.

The output dictionary is passed directly to the parallel workers in `run_worker_wrapper.py` and `worker_hadronic_gen.py`. There, the baseline masses are used to calculate the specific scaling factors required to generate a diverse population of hadronic stars that cover the observed mass gap, ensuring the synthetic dataset is physically robust.